<a href="https://colab.research.google.com/github/CalculatedContent/xgbwwdata/blob/main/XGBDataTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install xgbwwdata package

In [1]:
# Fresh clone
!rm -rf /content/repo_xgbwwdata
!git clone https://github.com/CalculatedContent/xgbwwdata.git /content/repo_xgbwwdata

# MUST use %pip so install targets current kernel env
%pip install -U pip setuptools wheel
%pip install -r /content/repo_xgbwwdata/requirements.txt
%pip install -e /content/repo_xgbwwdata --no-build-isolation --no-deps

# Clear stale modules
import sys
for m in list(sys.modules):
    if m == "xgbwwdata" or m.startswith("xgbwwdata."):
        del sys.modules[m]

# Optional: ensure repo root isn't shadowing
sys.path = [p for p in sys.path if p not in ("/content/repo_xgbwwdata", "/content/xgbwwdata")]

import xgbwwdata
print("module:", xgbwwdata)
print("__file__:", getattr(xgbwwdata, "__file__", None))
print("__path__:", getattr(xgbwwdata, "__path__", None))
print("exports:", [x for x in dir(xgbwwdata) if not x.startswith("_")])

from xgbwwdata import Filters, scan_datasets, load_dataset
print("import OK")

Cloning into '/content/repo_xgbwwdata'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 4), reused 19 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 12.48 KiB | 12.48 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Obtaining file:///content/repo_xgbwwdata
  Checking if build backend supports build_editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for xgbwwdata (pyproject.toml) ... done
  Created wheel for xgbwwdata: filename=xgbwwdata-0.1.0-0.editable-py3-none-any.whl size=3222 sha256=1cb0e8feef05cf342689f3ec78c57a5581c0a2973915f94bede89b9202128f87
  Stored in directory: /tmp/pip-ephem-wheel-cache-tlcr8q15/wheels/e0/e1/e6/eeaf0c2327cae562e1882491ed06087db5851fceb169a0ae47
Successfully built xgbwwdata
  Attempting uninstall: xgbwwdata
    Found existing installation: xgbwwdata 0.1.0
    Uninstalling xgbwwdata-0.1

# SImple test: get list of openml datasets



In [2]:
!pip install -q  openml

from xgbwwdata import Filters, scan_datasets, load_dataset
import numpy as np

filters = Filters(
    min_rows=200,
    max_rows=2000,            # keep small for speed
    max_features=5000,
    max_dense_elements=5_000_000,
    preprocess=True,
)

# Scan a few datasets (OpenML only for speed)
df = scan_datasets(
    sources=["openml"],
    limit=3,
    filters=filters,
    smoke_train=True,
    random_state=0,
)

print("scan rows:", len(df))
display(df)

# Load the first dataset back by UID
uid = df.iloc[0]["dataset_uid"]
X, y, meta = load_dataset(uid, filters=filters)

print("\nLoaded:", uid)
print("X type:", type(X), "shape:", getattr(X, "shape", None))
print("y type:", type(y), "shape:", getattr(y, "shape", None))
print("meta:", meta)
print("unique y:", np.unique(y)[:10])

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['m' 'marvi' 'corr' 'jurofm' 's' 'p']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


scan rows: 3


,source,dataset_uid,name,task_type,n_rows,n_features,n_classes
0,openml,openml:2,anneal,classification,898,55,5
1,openml,openml:3,kr-vs-kp,classification,2000,73,2
2,openml,openml:5,arrhythmia,classification,452,343,13



Loaded: openml:2
X type: <class 'numpy.ndarray'> shape: (898, 55)
y type: <class 'numpy.ndarray'> shape: (898,)
meta: {'n_rows': 898, 'n_features': 55, 'n_classes': 5, 'dataset_uid': 'openml:2', 'name': 'anneal', 'task_type': 'classification'}
unique y: [0 1 2 3 4]


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['m' 'marvi' 'corr' 'jurofm' 's' 'p']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


# More extensive, check all sorces

In [ ]:
from xgbwwdata import Filters, scan_datasets

filters = Filters(
    min_rows=200,
    max_rows=60000,
    max_features=50000,
    max_dense_elements=int(2e8),
    preprocess=True,
)

df = scan_datasets(
    sources=["openml", "pmlb", "keel", "libsvm", "amlb"],  # default is all
    limit=200,              # stop after N passing datasets (None = all)
    filters=filters,
    smoke_train=True,       # 1-round XGBoost check
    random_state=0,
)

print(df.head())
print(df["source"].value_counts())